In [1]:
# import required modules
import pandas as pd

In [2]:
# read input invoice file
invoice_df = pd.read_csv('dataset1.csv')
invoice_df.head()

,invoice_id,legal_entity,counter_party,rating,status,value
0,1,L1,C1,1,ARAP,10
1,2,L2,C2,2,ARAP,20
2,3,L3,C3,4,ACCR,30
3,4,L1,C4,6,ARAP,40
4,5,L2,C5,4,ACCR,50


In [3]:
# read input counterparty file
counter_party_df = pd.read_csv('dataset2.csv')
counter_party_df.head()

,counter_party,tier
0,C1,1
1,C2,2
2,C3,3
3,C4,4
4,C5,5


In [4]:
# join invoice and counterparty datasets
joined_df = invoice_df.merge(counter_party_df, on='counter_party', how='left')
joined_df.head()

,invoice_id,legal_entity,counter_party,rating,status,value,tier
0,1,L1,C1,1,ARAP,10,1
1,2,L2,C2,2,ARAP,20,2
2,3,L3,C3,4,ACCR,30,3
3,4,L1,C4,6,ARAP,40,4
4,5,L2,C5,4,ACCR,50,5


In [5]:
# use pivot table to calculate max rating per category
pivot_rating = pd.pivot_table(
    joined_df, 
    index=["counter_party"], 
    values=["rating"], 
    aggfunc={"rating": max}
)

# rename column
pivot_rating.columns = ['max_rating_counter_party']

# convert pivot table to dataframe
pivot_rating_df = pivot_rating.reset_index()

# add column max_rating_counter_party to joined_df
joined_df = joined_df.merge(pivot_rating_df, on='counter_party', how='left')
joined_df.head()

,invoice_id,legal_entity,counter_party,rating,status,value,tier,max_rating_counter_party
0,1,L1,C1,1,ARAP,10,1,3
1,2,L2,C2,2,ARAP,20,2,3
2,3,L3,C3,4,ACCR,30,3,6
3,4,L1,C4,6,ARAP,40,4,6
4,5,L2,C5,4,ACCR,50,5,6


In [6]:
# duplicate value column
joined_df['value2'] = joined_df['value']

# use pivot table to calculate sum value with status filters
pivot = pd.pivot_table(
    joined_df, 
    index=["legal_entity", "counter_party", "tier", "max_rating_counter_party"], 
    values=["value", "value2"], 
    aggfunc={
        "value": lambda x: x[joined_df["status"]=="ARAP"].sum(), 
        "value2": lambda x: x[joined_df["status"]=="ACCR"].sum()
    },
    margins=True,
    margins_name="Total"
)

# rename columns
pivot.columns = ['sum_value_ARAP', 'sum_value_ACCR']

# convert pivot table to dataframe
pivot_df = pivot.reset_index()

# calculate max rating for all data and impute into total row
max_rating_counter_party_all = max( joined_df['max_rating_counter_party'] )
pivot_df['max_rating_counter_party'] = pivot_df['max_rating_counter_party'].replace('', max_rating_counter_party_all)

# replace all empty strings with text 'Total'
pivot_df = pivot_df.replace('', 'Total')
pivot_df

,legal_entity,counter_party,tier,max_rating_counter_party,sum_value_ARAP,sum_value_ACCR
0,L1,C1,1,3,40,0
1,L1,C3,3,6,5,0
2,L1,C4,4,6,40,100
3,L2,C2,2,3,20,40
4,L2,C3,3,6,0,52
5,L2,C5,5,6,1000,115
6,L3,C3,3,6,0,145
7,L3,C6,6,6,145,60
8,Total,Total,Total,6,1250,512


In [7]:
# function to calculate sum value with status filters for a specific attribute
def pivot_grp( grp_name ): 
    
    # use pivot table to calculate sum value with status filters
    pivot_subtotal = pd.pivot_table(
        joined_df, index=[grp_name], 
        values=["rating", "value", "value2"], 
        aggfunc={
            "rating": max, 
            "value": lambda x: x[joined_df["status"]=="ARAP"].sum(), 
            "value2": lambda x: x[joined_df["status"]=="ACCR"].sum()
        }
    )

    # Rename the columns to match the desired output
    pivot_subtotal.columns = ['max_rating_counter_party', 'sum_value_ARAP', 'sum_value_ACCR']
    
    # convert pivot table to dataframe
    pivot_subtotal_df = pivot_subtotal.reset_index()
    return(pivot_subtotal_df)

# calculate total for each of legal entity, counterparty & tier
tier_df = pivot_grp('tier')
counter_party_df = pivot_grp('counter_party')
legal_entity_df = pivot_grp('legal_entity')

In [8]:
# combine all dataframes for desired output
output_df = pd.concat([pivot_df,legal_entity_df,counter_party_df,tier_df])

# replace all null values with text 'Total'
output_df = output_df.fillna("Total")

# convert pivot table to dataframe
output_df.reset_index(inplace=True,drop=True)
output_df

,legal_entity,counter_party,tier,max_rating_counter_party,sum_value_ARAP,sum_value_ACCR
0,L1,C1,1,3,40,0
1,L1,C3,3,6,5,0
2,L1,C4,4,6,40,100
3,L2,C2,2,3,20,40
4,L2,C3,3,6,0,52
5,L2,C5,5,6,1000,115
6,L3,C3,3,6,0,145
7,L3,C6,6,6,145,60
8,Total,Total,Total,6,1250,512
9,L1,Total,Total,6,85,100


In [9]:
# generate required output file
output_df.to_csv( 'output_pd.csv', index=False )